# Evolution Callbacks in DidgeLab

This notebook explains the callback system used by `Nuevolution` to monitor and control evolution runs.

**Topics:**
- The two direct callback slots: `callback_generation_ended` and `callback_loss_progress`
- The pub/sub event system (`generation_ended`, `generation_started`, `evolution_ended`)
- Helper callbacks: `create_loss_progress_callback`, `TqdmLossProgressCallback`
- Built-in monitors: `EvolutionMonitor`, `SaveEvolution`, `PrintEvolutionInformation`, `EarlyStopping`

## Setup

Import the evolution components and set up a minimal run for demonstration.

In [ ]:
import sys
sys.path.insert(0, "..")

from didgelab import (
    Nuevolution,
    TestLossFunction,
    GeoGenomeA,
)
from didgelab.app import init_app, get_app
from didgelab.evo.callbacks import (
    create_loss_progress_callback,
    TqdmLossProgressCallback,
    EvolutionMonitor,
    SaveEvolution,
    PrintEvolutionInformation,
    EarlyStopping,
)

# Initialize app (creates output folder, logging; required for pub/sub)
init_app("evolution_callbacks_example", create_output_folder=False)

## 2b. Using the modular loss API (`CompositeTairuaLoss`)

You can build a loss from reusable components (e.g. `FrequencyTuningLoss`, `ScaleTuningLoss`, `PeakAmplitudeLoss`) instead of `TestLossFunction`. The composite loss uses real acoustical simulation and returns a dict with a `"total"` key, so it works as a drop-in for `Nuevolution`.

In [ ]:
import numpy as np
from didgelab import (
    Nuevolution,
    GeoGenomeA,
    CompositeTairuaLoss,
    FrequencyTuningLoss,
)

# Target drone ~73 Hz and first overtone ~147 Hz (log2 in Hz)
target_freqs_log = np.log2(np.array([73.4, 146.8]))
modular_loss = CompositeTairuaLoss(max_error=5.0)
modular_loss.add_component("freq", FrequencyTuningLoss(target_freqs_log, [1.0, 1.0]))

evo = Nuevolution(
    modular_loss,
    GeoGenomeA.build(5),
    num_generations=2,
    population_size=4,
    generation_size=2,
    callback_generation_ended=lambda i, pop: print(f"Gen {i}: best total loss = {pop[0].loss['total']:.4f}"),
)
evo.evolve()

## 1. The Two Direct Callback Slots

`Nuevolution` exposes two optional callbacks that you assign directly:

### `callback_generation_ended`

Called **once per generation** after selection, with `(i_generation, population)`. The population is already sorted by loss (best first). Use this to log results, save checkpoints, update plots, or trigger early stopping.

### `callback_loss_progress`

Called **during** the loss computation step (`pool.map`) each time an individual's loss is computed. Receives `(i_generation, completed, total)` where `completed` is how many individuals have been evaluated so far in that generation. Use this for progress bars or streaming progress to a frontend.

## 2. Simple Custom Callbacks

Assign functions directly to the callback slots.

In [ ]:
def on_generation_ended(i_generation, population):
    best = population[0]
    print(f"Generation {i_generation}: best loss = {best.loss['total']:.4f}")

def on_loss_progress(i_generation, completed, total):
    pct = 100 * completed / total
    print(f"  gen {i_generation}: {completed}/{total} ({pct:.0f}%)", end="\r")

evo = Nuevolution(
    TestLossFunction(),
    GeoGenomeA.build(5),
    num_generations=3,
    population_size=6,
    generation_size=4,
    callback_generation_ended=on_generation_ended,
    callback_loss_progress=on_loss_progress,
)
population = evo.evolve()

## 3. Throttled Loss Progress with `create_loss_progress_callback`

During loss computation, callbacks can fire very frequently. `create_loss_progress_callback` wraps your handler and throttles calls (default: at most every 0.1 seconds) to avoid flooding. Useful for UI updates or streaming to a frontend.

In [ ]:
progress_log = []

def record_progress(i_generation, completed, total):
    progress_log.append((i_generation, completed, total))

throttled = create_loss_progress_callback(
    record_progress,
    throttle_interval=0.05,  # min seconds between calls
)

evo = Nuevolution(
    TestLossFunction(),
    GeoGenomeA.build(5),
    num_generations=2,
    population_size=4,
    generation_size=2,
    callback_loss_progress=throttled,
)
evo.evolve()

print(f"Recorded {len(progress_log)} progress updates (throttled)")

## 4. Tqdm Progress Bar via `TqdmLossProgressCallback`

`TqdmLossProgressCallback` provides a tqdm progress bar that advances as each individual's loss is computed. Create an instance with the `Nuevolution` object and assign it to `callback_loss_progress`.

In [ ]:
evo = Nuevolution(
    TestLossFunction(),
    GeoGenomeA.build(5),
    num_generations=2,
    population_size=4,
    generation_size=2,
)
evo.callback_loss_progress = TqdmLossProgressCallback(evo)
evo.evolve()

## 5. Pub/Sub Events

Internally, `Nuevolution` publishes events on the app's pub/sub bus:

- **`generation_started`** — `(i_generation, population)` before computing losses for offspring
- **`generation_ended`** — `(i_generation, population)` after selection (population sorted by loss)
- **`evolution_ended`** — `(population)` when evolution completes
- **`log_evolution_operations`** — `(i_generation, population, offspring, losses)` for detailed logging

You can subscribe to these events without modifying the evolution runner. This is how `NuevolutionProgressBar`, `PrintEvolutionInformation`, `EarlyStopping`, and writers like `NuevolutionWriter` hook in.

In [ ]:
from tqdm import tqdm

pbar = None

def on_gen_ended(i_generation, population):
    global pbar
    if pbar is None:
        evo = get_app().get_service(Nuevolution)
        pbar = tqdm(total=evo.num_generations, desc="evolution")
    pbar.update(1)
    best = population[0].loss["total"]
    pbar.set_postfix(best_loss=f"{best:.3f}")

get_app().subscribe("generation_ended", on_gen_ended)

evo = Nuevolution(
    TestLossFunction(),
    GeoGenomeA.build(5),
    num_generations=3,
    population_size=4,
    generation_size=2,
)
evo.evolve()
if pbar:
    pbar.close()

## 6. Built-in Monitors Overview

| Class | Purpose | How it hooks in |
|-------|---------|-----------------|
| `EvolutionMonitor` | Plots bore, impedance, notes each generation | `callback_generation_ended` (via `init_standard_evolution`) |
| `SaveEvolution` | Writes population & losses to `.json.gz` | `callback_generation_ended` (via `init_standard_evolution`) |
| `NuevolutionProgressBar` | tqdm bar per generation | `subscribe("generation_ended")` |
| `PrintEvolutionInformation` | Logs loss & notes every N generations | `subscribe("generation_ended")` |
| `EarlyStopping` | Sets `evo.continue_evolution = False` when no improvement | `subscribe("generation_ended")` |

`init_standard_evolution(target_freqs, evo)` wires `EvolutionMonitor` and `SaveEvolution` to `evo.callback_generation_ended`. It expects an acoustical loss and target frequencies. See the tutorial notebooks for full inverse-design examples.

## 7. Early Stopping Example

`EarlyStopping` subscribes to `generation_ended` and checks whether the best loss has improved. If the loss has not improved for `duration` generations, it sets `evo.continue_evolution = False` and evolution stops gracefully.

In [ ]:
# EarlyStopping requires get_app().get_service(Nuevolution) to work,
# so we must create evo first and register it.
evo = Nuevolution(
    TestLossFunction(),
    GeoGenomeA.build(5),
    num_generations=100,  # we'll stop earlier
    population_size=6,
    generation_size=4,
)

# Stop if no improvement for 3 generations
EarlyStopping(duration=3)

population = evo.evolve()
print(f"Stopped after {evo.i_generation} generations")

## 8. PrintEvolutionInformation

Logs the best individual's loss breakdown and note analysis every `interval` generations (and always at generation 1). Useful for understanding how the loss components and tuning evolve.

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

evo = Nuevolution(
    TestLossFunction(),
    GeoGenomeA.build(5),
    num_generations=4,
    population_size=4,
    generation_size=2,
)

# Log every 2 generations (and gen 1)
PrintEvolutionInformation(interval=2, base_freq=440)

evo.evolve()

## Summary

- **Direct callbacks**: Assign `callback_generation_ended` and/or `callback_loss_progress` when creating `Nuevolution` or afterward.
- **Throttling**: Use `create_loss_progress_callback` to avoid flooding during loss computation.
- **tqdm bars**: Use `TqdmLossProgressCallback` for per-individual progress, or subscribe to `generation_ended` for per-generation progress.
- **Pub/sub**: Subscribe to `generation_ended`, `generation_started`, `evolution_ended` for cross-cutting behavior (logging, early stopping, writers).
- **Built-ins**: `EvolutionMonitor`, `SaveEvolution`, `PrintEvolutionInformation`, `EarlyStopping` combine via `init_standard_evolution` or used individually.